# 🌐 Notebook 02 - Alat untuk Visualisasi Geospasial

**Selamat datang di Notebook 02!** Setelah belajar peta statis di Notebook 01, sekarang kita akan membuat **peta interaktif** yang bisa di-zoom, di-klik, dan lebih dinamis!

---

## 🎯 Tujuan Pembelajaran

Setelah menyelesaikan notebook ini, Anda akan dapat:
1. Membuat **peta interaktif** menggunakan Folium
2. Menambahkan **marker & popup** dengan informasi detail
3. Membuat **scatter map** dengan Plotly
4. Memahami **perbedaan** antara berbagai alat visualisasi
5. Memilih **alat yang tepat** untuk kebutuhan Anda

⏱️ **Estimasi waktu:** 45-60 menit

---

## 📚 Apa itu Peta Interaktif?

**Perbedaan peta statis vs interaktif:**

| Fitur | Peta Statis (GeoPandas) | Peta Interaktif (Folium/Plotly) |
|-------|------------------------|----------------------------------|
| **Zoom** | ❌ Tidak bisa | ✅ Bisa zoom in/out |
| **Pan** | ❌ Tidak bisa geser | ✅ Bisa drag & geser |
| **Klik** | ❌ Tidak ada interaksi | ✅ Popup/tooltip saat klik/hover |
| **Format** | PNG/JPG | HTML (bisa dibuka di browser) |
| **Use case** | Laporan, paper, presentasi | Web, dashboard, eksplorasi data |

> **💡 TIP:** Gunakan peta statis untuk dokumen cetak, gunakan peta interaktif untuk web!

---

## 🛠️ Persiapan: Load Data Fasilitas Publik

Kita akan menggunakan data **fasilitas publik di Kabupaten Jember** seperti sekolah, rumah sakit, pasar, tempat wisata, dll. Data ini lebih masuk akal untuk visualisasi geospasial karena:
- ✅ Lokasi real di Kabupaten Jember
- ✅ Kategori yang beragam dan relevan
- ✅ Memiliki atribut seperti kapasitas, rating, deskripsi

In [1]:
# Import library
import pandas as pd
from pathlib import Path
import os

# Setup path (kompatibel dengan berbagai working directory)
if 'notebooks' in os.getcwd():
    ROOT = Path('..')
else:
    ROOT = Path('.')

CSV_FASILITAS = ROOT / 'data' / 'raw' / 'fasilitas_jember.csv'

# Load data
df = pd.read_csv(CSV_FASILITAS)

print(f'✅ Data fasilitas berhasil dimuat: {len(df)} baris')
print(f'\n📊 Kolom yang tersedia:')
print(df.columns.tolist())
print('\n📋 Kategori fasilitas:')
print(df['kategori'].value_counts())
print('\n📋 Preview data:')
df.head(10)

✅ Data fasilitas berhasil dimuat: 20 baris

📊 Kolom yang tersedia:
['id', 'nama', 'kategori', 'kecamatan', 'lat', 'lon', 'kapasitas', 'rating', 'deskripsi']

📋 Kategori fasilitas:
kategori
pendidikan      5
wisata          5
ekonomi         4
kesehatan       3
ruang_publik    2
transportasi    1
Name: count, dtype: int64

📋 Preview data:


,id,nama,kategori,kecamatan,lat,lon,kapasitas,rating,deskripsi
0,1,SMAN 1 Jember,pendidikan,Kaliwates,-8.1650,113.6995,1200,4.5,Sekolah menengah favorit di pusat kota
1,2,RSUD dr. Soebandi,kesehatan,Patrang,-8.1734,113.6801,450,4.2,Rumah sakit rujukan utama Jember
2,3,Universitas Jember,pendidikan,Sumbersari,-8.1652,113.7220,35000,4.6,Kampus universitas negeri terbesar
3,4,Pasar Tanjung,ekonomi,Kaliwates,-8.1689,113.6945,500,4.0,Pasar tradisional pusat kota
4,5,Papuma Beach,wisata,Wuluhan,-8.2789,113.8689,1000,4.7,Pantai pasir putih dengan tebing karang
5,6,Air Terjun Tancak,wisata,Panti,-8.2145,113.3789,300,4.4,Air terjun dengan kolam alami
6,7,SMP Negeri 1 Jember,pendidikan,Kaliwates,-8.1705,113.7034,900,4.3,Sekolah menengah pertama favorit
7,8,Puskesmas Balung,kesehatan,Balung,-8.2911,113.9081,150,3.9,Pusat kesehatan masyarakat
8,9,Alun-alun Jember,ruang_publik,Kaliwates,-8.1672,113.6978,2000,4.1,Ruang terbuka publik pusat kota
9,10,Kampung Inggris,pendidikan,Rambipuji,-8.1445,113.6208,500,4.5,Lembaga kursus bahasa Inggris


---

## 🗺️ BAGIAN 1: Folium - Peta Interaktif Mudah

### Apa itu Folium?

**Folium** adalah library Python yang menghasilkan peta interaktif berbasis **Leaflet.js** (library JavaScript populer).

**Kelebihan:**
- ✅ Mudah digunakan (hanya beberapa baris kode)
- ✅ Gratis, tidak perlu API key untuk tiles standar
- ✅ Output HTML bisa langsung dibuka di browser
- ✅ Banyak plugin (heatmap, cluster, choropleth)

**Kekurangan:**
- ⚠️ Lambat jika marker terlalu banyak (>1000)
- ⚠️ Styling tidak sefleksibel Plotly

---

### Langkah 1: Buat Peta Dasar

Kita akan membuat peta berpusat pada Kabupaten Jember (rata-rata koordinat fasilitas):

In [2]:
import folium

# Hitung pusat peta (rata-rata lat & lon)
center_lat = df['lat'].mean()
center_lon = df['lon'].mean()
print(f'📍 Pusat peta: ({center_lat:.4f}, {center_lon:.4f})')

# Buat peta dasar
m = folium.Map(
    location=[center_lat, center_lon],  # koordinat pusat
    zoom_start=10,                      # level zoom (1=dunia, 18=detail jalan)
    tiles='CartoDB positron'            # style map (alternatif: 'OpenStreetMap', 'Stamen Terrain')
)

print('✅ Peta dasar berhasil dibuat!')
print(f'📍 Zoom level: 10 (cocok untuk tingkat kabupaten)')
print('Tunggu sebentar, kita akan tambahkan marker...')

📍 Pusat peta: (-8.1903, 113.6571)
✅ Peta dasar berhasil dibuat!
📍 Zoom level: 10 (cocok untuk tingkat kabupaten)
Tunggu sebentar, kita akan tambahkan marker...


> **💡 PENJELASAN:**
> - `location=[lat, lon]` → koordinat pusat peta
> - `zoom_start` → angka 1-18 (makin besar makin detail)
> - `tiles` → gaya basemap (coba: 'OpenStreetMap', 'Stamen Toner')

---

### Langkah 2: Tambahkan Marker dengan Popup

Kita akan menambahkan **CircleMarker** untuk setiap fasilitas, dengan warna berdasarkan kategori:

In [3]:
# Buat ulang peta (agar bersih)
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='CartoDB positron')

# Definisikan warna untuk setiap kategori fasilitas
palette = {
    'pendidikan': '#3498db',      # biru
    'kesehatan': '#e74c3c',       # merah
    'wisata': '#2ecc71',          # hijau
    'ekonomi': '#f39c12',         # orange
    'ruang_publik': '#9b59b6',    # ungu
    'transportasi': '#34495e'     # abu gelap
}

# Icon untuk setiap kategori
icons = {
    'pendidikan': '🎓',
    'kesehatan': '🏥',
    'wisata': '🏖️',
    'ekonomi': '🏪',
    'ruang_publik': '🏛️',
    'transportasi': '🚌'
}

# Loop setiap fasilitas dan tambahkan marker
for idx, row in df.iterrows():
    # Ambil warna dan icon berdasarkan kategori
    color = palette.get(row['kategori'], '#95a5a6')  # default abu-abu
    icon = icons.get(row['kategori'], '📍')
    
    # Tentukan ukuran marker berdasarkan rating
    radius = 6 + (row['rating'] * 2)  # rating tinggi = marker lebih besar
    
    # Buat konten popup (HTML)
    popup_html = f"""
    <div style='font-family: Arial; min-width: 200px;'>
        <h4 style='margin-bottom: 5px; color: {color};'>{icon} {row['nama']}</h4>
        <hr style='margin: 5px 0;'>
        <b>📂 Kategori:</b> {row['kategori'].title()}<br>
        <b>📍 Kecamatan:</b> {row['kecamatan']}<br>
        <b>⭐ Rating:</b> {row['rating']}/5.0<br>
        <b>👥 Kapasitas:</b> {row['kapasitas']:,} orang<br>
        <br>
        <i style='color: #7f8c8d; font-size: 11px;'>{row['deskripsi']}</i>
    </div>
    """
    
    # Tambahkan CircleMarker
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,               # ukuran berdasarkan rating
        color=color,                 # warna border
        fill=True,
        fill_color=color,            # warna isi
        fill_opacity=0.7,
        weight=2,
        popup=folium.Popup(popup_html, max_width=350)
    ).add_to(m)

print(f'✅ {len(df)} marker fasilitas berhasil ditambahkan!')
print('🔍 Klik marker untuk lihat detail fasilitas')
print(f'🎨 Warna: Pendidikan (biru), Kesehatan (merah), Wisata (hijau), Ekonomi (orange), dll')

✅ 20 marker fasilitas berhasil ditambahkan!
🔍 Klik marker untuk lihat detail fasilitas
🎨 Warna: Pendidikan (biru), Kesehatan (merah), Wisata (hijau), Ekonomi (orange), dll


> **💡 TIPS FOLIUM:**
> - `CircleMarker` vs `Marker`: CircleMarker lebih ringan dan konsisten di semua zoom level
> - Popup bisa pakai **HTML**, jadi Anda bisa tambah gambar, link, styling!
> - `fill_opacity` → transparansi (0.5 = semi-transparan)

---

### Langkah 3: Simpan & Tampilkan Peta

In [4]:
# Simpan ke file HTML
output_path = ROOT / 'exports' / 'map_fasilitas_folium.html'
output_path.parent.mkdir(parents=True, exist_ok=True)  # buat folder exports jika belum ada
m.save(str(output_path))
print(f'💾 Peta disimpan ke: {output_path}')
print(f'🌐 Buka file HTML di browser untuk melihat peta interaktif!')

# Tampilkan peta di Jupyter (jika dijalankan di Jupyter)
m

💾 Peta disimpan ke: ../exports/map_fasilitas_folium.html
🌐 Buka file HTML di browser untuk melihat peta interaktif!


> **🎯 COBA SEKARANG:**
> 1. Buka file `exports/map_fasilitas_folium.html` di browser
> 2. Zoom in/out dengan scroll mouse
> 3. Klik marker untuk lihat popup dengan detail fasilitas
> 4. Drag untuk menggeser peta
> 5. Perhatikan ukuran marker yang berbeda berdasarkan rating!

---

## 📊 BAGIAN 2: Plotly - Visualisasi Interaktif Modern

### Apa itu Plotly?

**Plotly** adalah library visualisasi modern dengan fitur interaktif canggih.

**Kelebihan:**
- ✅ Styling sangat fleksibel
- ✅ Terintegrasi dengan dashboard (Dash)
- ✅ Animasi & interaksi smooth
- ✅ Performa baik untuk ribuan titik

**Kekurangan:**
- ⚠️ Beberapa style Mapbox butuh token (gratis, tapi perlu daftar)
- ⚠️ Sedikit lebih kompleks daripada Folium

---

### Membuat Scatter Mapbox dengan Plotly

In [5]:
import plotly.express as px

# Coba gunakan 'carto-positron' (tidak perlu token)
# Jika gagal, otomatis fallback ke 'open-street-map'
map_style = 'carto-positron'

print('🎨 Membuat peta Plotly...')

try:
    fig = px.scatter_mapbox(
        df,
        lat='lat',                    # kolom latitude
        lon='lon',                    # kolom longitude
        color='kategori',             # warna berdasarkan kategori
        size='rating',                # ukuran titik berdasarkan rating
        hover_name='nama',            # judul tooltip
        hover_data={
            'lat': False, 
            'lon': False, 
            'kecamatan': True, 
            'rating': ':.1f',         # format 1 desimal
            'kapasitas': ':,',        # format dengan koma pemisah ribuan
            'deskripsi': True
        },
        color_discrete_sequence=px.colors.qualitative.Set2,  # palet warna
        zoom=9,
        mapbox_style=map_style,
        title='📍 Peta Fasilitas Publik Kabupaten Jember'
    )
    
    # Sesuaikan layout
    fig.update_layout(
        height=650,
        margin={'r': 0, 't': 60, 'l': 0, 'b': 0},
        font=dict(family='Arial', size=12),
        title_font_size=16
    )
    
    # Sesuaikan ukuran marker
    fig.update_traces(marker=dict(sizemode='diameter', sizemin=8, sizeref=0.1))
    
    print('✅ Peta Plotly berhasil dibuat!')
    print('🖱️ Hover di titik untuk melihat detail fasilitas')
    fig.show()
    
except Exception as e:
    print(f'⚠️ Gagal dengan style {map_style}. Error: {e}')
    print('🔄 Mencoba dengan open-street-map...')
    
    fig = px.scatter_mapbox(
        df, lat='lat', lon='lon', color='kategori', size='rating',
        hover_name='nama', zoom=9, mapbox_style='open-street-map',
        title='📍 Peta Fasilitas Publik Kabupaten Jember'
    )
    fig.update_layout(height=650)
    print('✅ Peta dengan OSM tiles berhasil!')
    fig.show()

🎨 Membuat peta Plotly...


/tmp/ipykernel_1428/2749522973.py:10: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(


✅ Peta Plotly berhasil dibuat!
🖱️ Hover di titik untuk melihat detail fasilitas


> **💡 PENJELASAN PLOTLY:**
> - `size='rating'` → ukuran bubble proporsional dengan rating fasilitas
> - `hover_data` → kontrol kolom apa yang muncul di tooltip (format bisa dikustomisasi)
> - `color_discrete_sequence` → palet warna kustom untuk kategori
> - `sizeref=0.1` → skala ukuran marker agar tidak terlalu besar/kecil
> - Coba **hover** di titik untuk lihat tooltip dengan informasi lengkap!

---

### Simpan Peta Plotly ke HTML

In [6]:
# Simpan figure Plotly ke HTML
plotly_output = ROOT / 'exports' / 'map_fasilitas_plotly.html'
plotly_output.parent.mkdir(parents=True, exist_ok=True)
fig.write_html(str(plotly_output))
print(f'💾 Peta Plotly disimpan ke: {plotly_output}')
print(f'📊 File HTML dapat dibuka di browser untuk eksplorasi interaktif')

💾 Peta Plotly disimpan ke: ../exports/map_fasilitas_plotly.html
📊 File HTML dapat dibuka di browser untuk eksplorasi interaktif


---

## 📊 PERBANDINGAN LENGKAP ALAT VISUALISASI

### Tabel Perbandingan Detail

| Alat | Tipe | Kelebihan | Kekurangan | Cocok untuk | Gratis? |
|------|------|-----------|------------|-------------|--------|
| **GeoPandas** | Kode (Python) | • Terintegrasi Pandas<br>• Analisis spasial<br>• Output PNG/SVG | • Tidak interaktif<br>• Styling terbatas | Analisis data, peta statis, paper | ✅ |
| **Folium** | Kode (Python) | • Mudah dipelajari<br>• HTML interaktif<br>• Banyak plugin | • Lambat >1000 marker<br>• Styling terbatas | Peta web sederhana, dashboard | ✅ |
| **Plotly** | Kode (Python) | • Interaksi smooth<br>• Styling fleksibel<br>• Performa baik | • Butuh token Mapbox (opsional)<br>• Kurva belajar | Dashboard, web app, analisis interaktif | ✅ (token gratis) |
| **QGIS** | GUI (Desktop) | • Fitur GIS lengkap<br>• Plugin banyak<br>• Analisis kompleks | • Aplikasi berat<br>• Kurva belajar tinggi | Analisis spasial profesional | ✅ |
| **Kepler.gl** | GUI (Web) | • Visualisasi 3D<br>• Big data<br>• Animasi temporal | • Perlu data terstruktur<br>• Kurang cocok analisis | Visualisasi big data, presentasi | ✅ |

---

### 🤔 Kapan Menggunakan Apa?

**Gunakan GeoPandas jika:**
- Anda perlu analisis data (join, buffer, intersection)
- Output untuk paper/laporan (PDF/PNG)
- Terintegrasi dengan workflow Pandas

**Gunakan Folium jika:**
- Perlu peta interaktif cepat (<500 marker)
- Output untuk web/blog sederhana
- Tidak perlu styling rumit

**Gunakan Plotly jika:**
- Membuat dashboard profesional
- Perlu styling & interaksi canggih
- Data > 1000 titik

**Gunakan QGIS jika:**
- Analisis spasial kompleks (buffer, overlay, routing)
- Perlu editing geometri manual
- Membuat peta kartografi profesional

**Gunakan Kepler.gl jika:**
- Visualisasi jutaan titik
- Animasi temporal (time-series)
- Presentasi WOW-factor tinggi

---

## ✍️ LATIHAN MANDIRI

### **Latihan 1: Filter Fasilitas Berdasarkan Rating** (mudah)

Buat peta Folium yang hanya menampilkan fasilitas dengan rating ≥ 4.0

**Hint:** Gunakan `df[df['rating'] >= 4.0]` untuk filter data

---

### **Latihan 2: Peta Kategori Spesifik** (menengah)

Buat peta Plotly yang hanya menampilkan fasilitas kategori **wisata** dan **pendidikan**

**Hint:** Gunakan `df[df['kategori'].isin(['wisata', 'pendidikan'])]`

---

### **Latihan 3: Marker dengan Icon Kustom** (advanced)

Modifikasi peta Folium untuk menggunakan `folium.Marker` dengan icon Font Awesome yang berbeda untuk setiap kategori

**Hint:** Lihat dokumentasi Folium untuk `Icon` parameter

---

> **📝 Silakan coba sebelum lihat jawaban!**

## 📋 Jawaban Latihan

### Jawaban Latihan 1: Filter Rating Tinggi

In [7]:
# Latihan 1: Hanya fasilitas dengan rating ≥ 4.0
df_high_rating = df[df['rating'] >= 4.0]

m_filtered = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='CartoDB positron')

for idx, row in df_high_rating.iterrows():
    color = palette.get(row['kategori'], '#95a5a6')
    
    popup_html = f"""
    <div style='font-family: Arial;'>
        <h4 style='color: {color};'>{row['nama']}</h4>
        <b>⭐ Rating:</b> {row['rating']}/5.0<br>
        <b>📂 Kategori:</b> {row['kategori']}<br>
        <i>✨ Fasilitas rating tinggi</i>
    </div>
    """
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=10,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=popup_html
    ).add_to(m_filtered)

print(f'✅ Menampilkan {len(df_high_rating)} fasilitas dengan rating ≥ 4.0')
m_filtered

✅ Menampilkan 16 fasilitas dengan rating ≥ 4.0


### Jawaban Latihan 2: Filter Kategori Wisata & Pendidikan

In [9]:
# Latihan 2: Hanya kategori wisata dan pendidikan
df_subset = df[df['kategori'].isin(['wisata', 'pendidikan'])]

fig_subset = px.scatter_mapbox(
    df_subset,
    lat='lat',
    lon='lon',
    color='kategori',
    size='rating',
    hover_name='nama',
    hover_data={'lat': False, 'lon': False, 'kecamatan': True, 'rating': ':.1f'},
    zoom=9,
    mapbox_style='carto-positron',
    title=f'📍 Fasilitas Wisata & Pendidikan di Jember ({len(df_subset)} lokasi)'
)

fig_subset.update_layout(height=600)
print(f'✅ Menampilkan {len(df_subset)} fasilitas (wisata & pendidikan)')
fig_subset.show()
        'jenis': False,    # sembunyikan
        'kota': True,      # tampilkan
        'skor': True       # tampilkan
    },
    zoom=6.5,
    mapbox_style='carto-positron',
    title='Peta UMKM - Tooltip Kustom'
)

fig_custom.update_layout(height=600)
print('✅ Peta dengan tooltip kustom berhasil!')
fig_custom.show()

IndentationError: unindent does not match any outer indentation level (<string>, line 23)

---

## 🎉 Selamat!

Anda telah menyelesaikan Notebook 02 dan mempelajari:
- ✅ Membuat peta interaktif dengan **Folium** (marker + popup)
- ✅ Membuat scatter map dengan **Plotly**
- ✅ Memahami **perbedaan** berbagai alat visualisasi
- ✅ Memilih **alat yang tepat** untuk kebutuhan
- ✅ Menyimpan peta ke **HTML**

---

## 🚀 Lanjut ke Notebook 03

Di notebook terakhir, Anda akan:
- Mengerjakan **tugas praktikum** lengkap
- Membuat peta dengan **marker cluster**
- Agregasi data per kecamatan
- Mendapat **rubrik penilaian**

**Buka:** `03_latihan_tugas.ipynb`

---

> **💡 TIPS LANJUTAN:**
> - Eksplorasi plugin Folium: HeatMap, MarkerCluster, Choropleth
> - Pelajari Dash (framework Plotly) untuk dashboard real-time
> - Gabungkan GeoPandas + Folium untuk analisis + visualisasi

# 02 - Alat untuk Visualisasi Geospasial

Notebook ini memperlihatkan demo singkat Folium (marker + popup) dan Plotly (scatter_mapbox). Kita akan memuat data UMKM dari `../data/raw/umkm_sample.csv` dan menampilkan peta interaktif dasar.

In [ ]:
# Imports dan path
import pandas as pd
from pathlib import Path

ROOT = Path('..') if Path('.').name == 'notebooks' else Path('.')
CSV_UMKM = ROOT / 'data' / 'raw' / 'umkm_sample.csv'
df = pd.read_csv(CSV_UMKM)
print('Loaded', len(df), 'rows')
df.head()

## Demo Folium - Marker dan Popup
Folium menggunakan tiles dari penyedia seperti CartoDB/ OpenStreetMap dan bekerja offline for tiles if you have them cached; no API keys are required for common tiles like 'CartoDB positron' or 'OpenStreetMap'.

In [ ]:
import folium
# Buat peta berpusat pada rata-rata koordinat
center = [df['lat'].mean(), df['lon'].mean()]
m = folium.Map(location=center, zoom_start=7, tiles='CartoDB positron')

# warna sederhana berdasarkan jenis
palette = {'makanan':'red','kue':'purple','jasa':'blue','kerajinan':'green','minuman':'orange','retail':'darkgreen','jasaha':'gray'}

for _, r in df.iterrows():
    color = palette.get(r.get('jenis',''), 'black')
    popup_html = f"<b>{r.nama}</b><br>{r.kota} - {r.kecamatan}<br>Jenis: {r.jenis}<br>Skor: {r.skor}"
    folium.CircleMarker(location=[r.lat, r.lon], radius=6, color=color, fill=True, fill_color=color, popup=folium.Popup(popup_html, max_width=250)).add_to(m)

# Simpan ke exports agar mudah dibuka
out = ROOT / 'exports' / 'map_umkm_folium.html'
m.save(out)
print('Saved Folium map to', out)
m  # in Jupyter this will render inline

## Demo Plotly - scatter_mapbox
Plotly dapat menggunakan Mapbox styles (e.g., 'carto-positron') or the built-in 'open-street-map' tiles. Using certain Mapbox styles may require a Mapbox access token; if you don't have one, use `mapbox_style='open-street-map'`.

In [ ]:
import plotly.express as px
# Jika tidak punya token, gunakan 'open-street-map' atau catatan tentang token
map_style = 'carto-positron'  # try; fallback to 'open-street-map' if you have no token
try:
    fig = px.scatter_mapbox(df, lat='lat', lon='lon', color='jenis', size='skor', hover_name='nama', zoom=6, mapbox_style=map_style)
    fig.update_layout(height=600)
    fig.show()
except Exception as e:
    print('Plotly map using mapbox style failed (no token?). Falling back to open-street-map. Error:', e)
    fig = px.scatter_mapbox(df, lat='lat', lon='lon', color='jenis', size='skor', hover_name='nama', zoom=6, mapbox_style='open-street-map')
    fig.update_layout(height=600)
    fig.show()

## Perbandingan singkat (GeoPandas / Folium / Plotly / QGIS / Kepler)
| Alat | Kelebihan | Kekurangan | Cocok untuk |
|-----|----------|-----------|------------|
| GeoPandas | Analisis & plotting statik, integrates with pandas | Tidak interaktif | Analisis data & peta statik |
| Folium | Peta interaktif, mudah menambahkan marker/popup | Kurang perform untuk banyak marker | Pembuatan peta interaktif cepat |
| Plotly | Interaksi, styling, integrasi web | Mapbox token untuk beberapa style | Visualisasi interaktif berbasis web |
| QGIS | GUI penuh fitur GIS | Kurva belajar, aplikasi berat | Analisis spasial mendalam |
| Kepler | Visualisasi besar & 3D | Memerlukan data terstruktur | Visualisasi skala besar |

## Latihan (2 soal)
1) Ubah warna marker Folium sehingga `jenis=='makanan'` tampilkan warna merah, `kue` ungu, lainnya biru. Simpan peta baru.
2) Di Plotly, ubah tooltip sehingga menampilkan `nama` dan `skor` saja, lalu ekspor figure ke HTML menggunakan `fig.write_html()`.

In [ ]:
# Jawaban singkat latihan
# 1) contoh perubahan warna sudah disiapkan di cell folium (ubah 'palette' jika ingin)
print('Untuk latihan Folium: sesuaikan palette dict lalu simpan ulang peta.')
# 2) contoh ekspor Plotly ke HTML
try:
    fig.write_html(ROOT / 'exports' / 'map_umkm_plotly.html')
    print('Saved Plotly map to exports/map_umkm_plotly.html')
except Exception as e:
    print('Could not save figure:', e)